In [ ]:
{
    "serves": [
        {
            "type": "First Serve",
            "sides": {
                "deuce": {
                    "wide": 20,
                    "body": 30,
                    "t": 50
                },
                "ad": {
                    "wide": 10,
                    "body": 20,
                    "t": 10
                }
            }
        },
        {
            "type": "Second Serve",
            "sides": {
                "deuce": {
                    "wide": 25,
                    "body": 30,
                    "t": 50
                },
                "ad": {
                    "wide": 15,
                    "body": 20,
                    "t": 10
                }
            }
        }
    ]
}

In [ ]:
import pandas as pd
import json

def label_zone(x):
    # wide, body, or t
    if x == '' or pd.isnull(x):
        return ''
    x = float(x)
    if x < -105 or x > 105:
        return 'wide'
    elif (-105 <= x <= -52.5) or (52.5 <= x <= 105):
        return 'body'
    elif -52.5 < x < 52.5:
        return 't'
    return ''

def scaled_x(stroke, x):
    # Scale x-coordinate for first or second serve
    if pd.isnull(x):
        return None
    if stroke == 'first_serve':
        return float(x) * 38.2764654418
    if stroke == 'second_serve':
        return float(x) * 38.2764654418
    return None

def extract_rudy_quan_serves(player_name):
    # Load file 
    player_name = 'Rudy Quan'
    file_path = f'../../data/mens/{player_name}/combined.xlsx'
    df = pd.read_excel(file_path, sheet_name='Shots')
    # Only keep serves by Rudy Quan
    serves = df[(df['player'] == 'Rudy Quan') & (df['stroke'].str.contains('serve', case=False, na=False))]
    # Check if 'side' column exists
    has_side = 'side' in serves.columns
    output = {"serves": []}
    for serve_type, serve_label in [('first_serve', 'First Serve'), ('second_serve', 'Second Serve')]:
        # Serve type
        serve_df = serves[serves['stroke'].str.lower() == serve_type]
        counts = {
            "deuce": {"wide": 0, "body": 0, "t": 0},
            "ad": {"wide": 0, "body": 0, "t": 0}
        }
        for idx, row in serve_df.iterrows():

            side = 'deuce'
            if has_side:
                val = str(row['side']).strip().lower()
                if val.startswith('a'):
                    side = 'ad'
                elif val.startswith('d'):
                    side = 'deuce'
            x = row.get('Bounce(x)', '')
            x_val = scaled_x(serve_type, x)
            zone = label_zone(x_val)
            if zone in ['wide', 'body', 't']:
                counts[side][zone] += 1
        output["serves"].append({
            "type": serve_label,
            "sides": counts
        })
    return json.dumps(output, indent=4)




### Cj Work

##### Helper Functions

In [1]:
import pandas as pd
import json
import numpy as np

In [2]:
def classify_zone(df):
    x = df['x_coord']
    y = df['y_coord']
    sign = x * y # if sign is pos, it's on ad side, if neg, it's deuce

    if (x < -105) or (x > 105):
        if sign > 0:
            side, zone = 'Ad', 'Wide'
        else:
            side, zone = 'Deuce', 'Wide'
    elif (-105 <= x <= -52.5) or (52.5 <= x <= 105):
        if sign > 0:
            side, zone = 'Ad', 'Body'
        else:
            side, zone = 'Deuce', 'Body'
    elif -52.5 < x < 52.5:
        if sign > 0:
            side, zone = 'Ad', 'T'
        else:
            side, zone = 'Deuce', 'T'
    else:
        side, zone = np.nan, np.nan
    
    return pd.Series({'side': side, 'Zone': zone})

In [4]:
# Load file 
player_name = 'Rudy Quan'
file_path = f'../../../data/mens/{player_name}/combined.xlsx'
df = pd.read_excel(file_path, sheet_name='Shots')


In [6]:
def serve_zone_bar(df, player_name):
    # Keep Player's Serve
    serves = df[(df['Player'] == player_name) & (df['Type'].isin(['first_serve', 'second_serve']))]

    # Keep only Serves In
    serves_in = serves[serves['Result'] == 'In'].copy()

    # Transform Coordinates
    serves_in.loc[:, 'x_coord'] = serves_in['Bounce (x)'] * 38.2764654418
    serves_in.loc[:, 'y_coord'] = (serves_in['Bounce (y)'] - 11.8872) * 38.2764654418

    # Create "side" and "serve_zone" columns
    serves_in[['side', 'serve_zone']] = serves_in.apply(classify_zone, axis=1)


    serves_in[['Type', 'side', 'serve_zone']].value_counts()

    # Step 1: Get the counts
    counts = serves_in[['Type', 'side', 'serve_zone']].value_counts().reset_index(name='count')

    # Step 2: Initialize output structure
    output = {'serves': []}

    # Step 3: Iterate through serve types
    for serve_type in counts['Type'].unique():
        serve_dict = {
            'type': 'First Serve' if serve_type == 'first_serve' else 'Second Serve',
            'sides': {
                'deuce': {'wide': 0, 'body': 0, 't': 0},
                'ad': {'wide': 0, 'body': 0, 't': 0}
            }
        }
        
        # Filter for this serve type
        subset = counts[counts['Type'] == serve_type]
        
        for _, row in subset.iterrows():
            side = row['side'].lower()     # 'Deuce' -> 'deuce'
            zone = row['serve_zone'].lower()  # 'Wide' -> 'wide'
            serve_dict['sides'][side][zone] = int(row['count'])
        
        output['serves'].append(serve_dict)

    # Save to current directory
    # with open('data/serve_zones.json', 'w') as f:
    #     json.dump(output, f, indent=4)

    return counts

In [7]:
serve_zone_bar(df, player_name)

,Type,side,serve_zone,count
0,first_serve,Deuce,Body,73
1,first_serve,Deuce,Wide,70
2,first_serve,Ad,T,69
3,first_serve,Deuce,T,68
4,first_serve,Ad,Body,57
5,first_serve,Ad,Wide,42
6,second_serve,Ad,Body,38
7,second_serve,Deuce,Body,29
8,second_serve,Ad,T,22
9,second_serve,Deuce,T,22


In [ ]:
serves_in[(serves_in['Type'] == 'first_serve') & (serves_in['side'] == 'Wide')]

In [ ]:
serves_in[(serves_in['side'] != 'Deuce') & (serves_in['side'] == 'Ad')][['Type', 'side', 'serve_zone']]

In [ ]:
def serve_zone_bar(player_name):
    # Only keep serves by Rudy Quan
    serves = df[(df['Player'] == 'Rudy Quan') & (df['Stroke'].str.contains('serve', case=False, na=False))]
    # Check if 'side' column exists
    has_side = 'side' in serves.columns
    output = {"serves": []}
    for serve_type, serve_label in [('first_serve', 'First Serve'), ('second_serve', 'Second Serve')]:
        # Serve type
        serve_df = serves[serves['Stroke'].str.lower() == serve_type]
        counts = {
            "deuce": {"wide": 0, "body": 0, "t": 0},
            "ad": {"wide": 0, "body": 0, "t": 0}
        }
        for idx, row in serve_df.iterrows():

            side = 'deuce'
            if has_side:
                val = str(row['side']).strip().lower()
                if val.startswith('a'):
                    side = 'ad'
                elif val.startswith('d'):
                    side = 'deuce'
            x = row.get('Bounce(x)', '')
            x_val = scaled_x(serve_type, x)
            zone = label_zone(x_val)
            if zone in ['wide', 'body', 't']:
                counts[side][zone] += 1
        output["serves"].append({
            "type": serve_label,
            "sides": counts
        })
    return json.dumps(output, indent=4)